In [1]:
import pandas as pd
import os
import codecs
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime


In [2]:
out_dir = './PROCESSED_DATA'
source ='./RAW_DATA/source.html'
sts_list = ['new','confirmed','updated','removed']
library ='./RAW_DATA/library.csv'
nav_x_path = '//nav/ul'

ldf = pd.read_csv(library,parse_dates = ['dateAdded'])
ldf['status'] = np.nan
ldf.head()


,name,path,level,dateAdded,status
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1.0,2019-06-21,NaN
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3.0,2019-06-21,NaN
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3.0,2019-06-14,NaN


In [3]:
df = pd.DataFrame()
soup = BeautifulSoup(codecs.open(source, 'r').read(), 'html.parser')
test = soup.nav.ul.find_all('a')

for child in soup.nav.ul.find_all('a'):
    df = df.append({'path':child.get('href'),
                    'name_source':child.get_text() ,
                    'level_source':child.get('data-level')},
                    ignore_index=True)

df.head()

,level_source,name_source,path
0,1,Home,/compXXX.com/wiki/home?authuser=0
1,1,Articles,/compXXX.com/wiki/articles?authuser=0
2,2,ML Use-Case ?s,/compXXX.com/wiki/articles/ml-use-case-s?authu...
3,2,Monitoring Performance of Appian Applications ...,/compXXX.com/wiki/articles/monitoring-performa...
4,2,Cross Env Code Mgmt,/compXXX.com/wiki/articles/cross-env-code-mgmt...


In [4]:
join_df = pd.merge(ldf, df, how='outer', on='path')
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1.0,2019-06-21,NaN,1,Community - PS Page (External)
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3.0,2019-06-21,NaN,3,Oracle UUID Generation - Reusable Snippet Library
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN,2,Accelerators & Foundations
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN,2,Appian Architecture
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3.0,2019-06-14,NaN,3,Application Playbook Guide - Industry Solution...


In [5]:
# set removed status
join_df.loc[join_df.name_source.isnull(), 'status'] = sts_list[3]
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1.0,2019-06-21,NaN,1,Community - PS Page (External)
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3.0,2019-06-21,NaN,3,Oracle UUID Generation - Reusable Snippet Library
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN,2,Accelerators & Foundations
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,NaN,2,Appian Architecture
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3.0,2019-06-14,NaN,3,Application Playbook Guide - Industry Solution...


In [6]:
# set new status

join_df.loc[join_df.name.isnull(), ['status','dateAdded']] = (sts_list[0],datetime.today().date())
join_df.level.fillna(join_df.level_source, inplace=True)
join_df.level_source.fillna(join_df.level, inplace=True)
join_df.name.fillna(join_df.name_source, inplace=True)
join_df.level = join_df.level.astype(np.int64)
join_df.level_source = join_df.level_source.astype(np.int64)
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1,2019-06-21,NaN,1,Community - PS Page (External)
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3,2019-06-21,NaN,3,Oracle UUID Generation - Reusable Snippet Library
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2,2019-06-14,NaN,2,Accelerators & Foundations
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2,2019-06-14,NaN,2,Appian Architecture
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3,2019-06-14,NaN,3,Application Playbook Guide - Industry Solution...


In [7]:
# set confirmed status
join_df.loc[(join_df.name == join_df.name_source) & 
            (join_df.level == join_df.level_source) & 
            (join_df.status != sts_list[0]), ['status']] = sts_list[1]

join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1,2019-06-21,confirmed,1,Community - PS Page (External)
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3,2019-06-21,confirmed,3,Oracle UUID Generation - Reusable Snippet Library
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2,2019-06-14,confirmed,2,Accelerators & Foundations
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2,2019-06-14,confirmed,2,Appian Architecture
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3,2019-06-14,confirmed,3,Application Playbook Guide - Industry Solution...


In [8]:
# set updated status
join_df.loc[join_df.status.isnull(), ['name','level']] = (np.nan, np.nan)
join_df.loc[join_df.status.isnull(), 'status'] = sts_list[2]
join_df.level.fillna(join_df.level_source, inplace=True)
join_df.name.fillna(join_df.name_source, inplace=True)
join_df.head()

,name,path,level,dateAdded,status,level_source,name_source
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1.0,2019-06-21,confirmed,1,Community - PS Page (External)
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3.0,2019-06-21,confirmed,3,Oracle UUID Generation - Reusable Snippet Library
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,confirmed,2,Accelerators & Foundations
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,confirmed,2,Appian Architecture
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3.0,2019-06-14,confirmed,3,Application Playbook Guide - Industry Solution...


In [9]:
join_df = join_df.drop(columns=['level_source', 'name_source'])
join_df.head()

,name,path,level,dateAdded,status
0,Community - PS Page (External),https://www.google.com/url?q=https%3A%2F%2Fcom...,1.0,2019-06-21,confirmed
1,Oracle UUID Generation - Reusable Snippet Library,/compXXX.com/wiki/libraries/reusable-snippet-l...,3.0,2019-06-21,confirmed
2,Accelerators & Foundations,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,confirmed
3,Appian Architecture,/compXXX.com/wiki/resources-being-merged-to-ar...,2.0,2019-06-14,confirmed
4,Application Playbook Guide - Industry Solution...,/compXXX.com/wiki/libraries/industry-solution-...,3.0,2019-06-14,confirmed
